# **DATA CLEANING**

## IMPORTS

In [1]:
# Imports
import numpy as np
import pandas as pd

## LOADING

In [43]:
# We load the data and check the first rows
data = pd.read_csv('../data/raw/idealista_viviendas_detalle.csv', )

# We change the names of the columns to make them easier to english and to avoid special characters
data.columns = ["url", "price", "zone","neighborhood", "built_area", "usable_area", "bedrooms", "bathrooms", "floor", "exterior", "elevator", "garage", "storage_room", "balcony", "new", "condition", "year", "heating", "agency", "consumption_value", "consumption_label", "emisions_value", "emissions_label", "description", "error"]
data.head()

,url,price,zone,neighborhood,built_area,usable_area,bedrooms,bathrooms,floor,exterior,...,condition,year,heating,agency,consumption_value,consumption_label,emisions_value,emissions_label,description,error
0,https://www.idealista.com/inmueble/109356873/,440.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",76.0,70.0,2.0,2.0,3.0,exterior,...,Buen estado,2025.0,Individual,ORDUNTE Inmobiliaria,23,A,4,A,"GASTOS DE GESTIÓN de 1,5 por ciento + IVA a ca...",NaN
1,https://www.idealista.com/inmueble/106221410/,442.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",82.0,NaN,2.0,2.0,1.0,exterior,...,NaN,NaN,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...,NaN
2,https://www.idealista.com/inmueble/107750109/,381.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",58.0,NaN,1.0,1.0,4.0,exterior,...,NaN,NaN,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...,NaN
3,https://www.idealista.com/inmueble/106221526/,575.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",104.0,NaN,3.0,2.0,1.0,exterior,...,NaN,NaN,Central,LOIOLA GESTIÓN INMOBILIARIA,NaN,A,NaN,A,La nueva promoción de Loiola se compone de 42 ...,NaN
4,https://www.idealista.com/inmueble/108491309/,306.000,deusto,"La Ribera-Ibarrekolanda, Bilbao",62.0,NaN,1.0,1.0,1.0,exterior,...,NaN,NaN,NaN,Loiola,NaN,A,NaN,A,"60 viviendas de obra nueva de 1, 2 y 3 habitac...",NaN


## CLEANING

In [44]:
# We check the summarize information of the dataset, to see the types of the columns and if there are null values
data.info()

# We remove the column that we will not use
data = data.drop(columns=["error"]) # Url is not useful for our analysis, but we will keep it for now in case we want to check some house manually

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                1490 non-null   object 
 1   price              1417 non-null   object 
 2   zone               1490 non-null   object 
 3   neighborhood       1417 non-null   object 
 4   built_area         1417 non-null   float64
 5   usable_area        968 non-null    float64
 6   bedrooms           1407 non-null   float64
 7   bathrooms          1415 non-null   float64
 8   floor              1271 non-null   float64
 9   exterior           1249 non-null   object 
 10  elevator           1392 non-null   object 
 11  garage             371 non-null    object 
 12  storage_room       381 non-null    object 
 13  balcony            780 non-null    object 
 14  new                1407 non-null   object 
 15  condition          1286 non-null   object 
 16  year               759 n

In [45]:
# We check the rows that contain all null values (except the zona column, which is the only one that has no null values)
null_columns = data[data.drop(columns=["zone", "url"]).isnull().all(axis=1)]
# The rows that contain all null values are the houses that have been removed from the website, probably because they 
# have been sold in just a day (the time between getting the url of the houses and getting the detailed information). 

# We get how many houses have been sold in one day per zone, just out of curiosity
print(null_columns['zone'].value_counts())

# As there are more houses in a zone than in others, we get the percentage of houses that have been sold in one day per zone
print((null_columns['zone'].value_counts() / data['zone'].value_counts()).sort_values(ascending=False))
# In some zones, more than 10% of the houses have been sold in one day.

# We drop these rows, as they do not provide any useful information
data = data.dropna(how='all', subset=data.columns.difference(['zone', "url"]))

zone
abando-albia             23
begona-santutxu          10
uribarri                  9
indautxu                  9
rekalde                   6
basurto-zorroza           5
ibaiondo                  5
casco-viejo               3
deusto                    2
otxarkoaga-txurdinaga     1
Name: count, dtype: int64
zone
begona-santutxu          0.113636
abando-albia             0.082143
uribarri                 0.066176
indautxu                 0.053892
casco-viejo              0.046154
basurto-zorroza          0.033113
rekalde                  0.028037
otxarkoaga-txurdinaga    0.027778
ibaiondo                 0.023585
deusto                   0.020619
san-adrian-la-pena            NaN
Name: count, dtype: float64


In [ ]:
# We check again the summarize information of the dataset
data.info()

# We check the percentage of null values per column
null_percentage = data.isnull().mean().sort_values(ascending=False)
print(null_percentage)

# We will tranform some columns to make them easier to use, changing their format, data type and filling null values.
# Let's go column by column:

#? Price: we will change the thousands separator and convert it to a numeric value (no null values)
data['price'] = data['price'].str.replace('.', '', regex=False).astype(float)

#? Zone and neighborhood: will transform them to categorical variables (no null values)
data['zone'] = data['zone'].astype('category')
data['neighborhood'] = data['neighborhood'].astype('category')

#? Built area: we will leave it as is, since there data type is correct (no null values)

#? Usable area: we will not do anything for now (null values: 32%)

#? Bedrooms: as there are just 10 null values, we will check them manually
data[data['bedrooms'].isnull()]["url"]
# All of them are studios where people can not live, so we do not take them into account and we will drop them
data = data.dropna(subset=['bedrooms'])
# We convert it to an integer. Finally (no null values)
data['bedrooms'] = data['bedrooms'].astype(int) 

# As we have drop some rows, we check the percentage and number of null values per column again
null_amount = data.isnull().sum().sort_values(ascending=False)
print(null_percentage, null_amount)

#? Bathrooms: will transform them to integer variables (no null values)
data['bathrooms'] = data['bathrooms'].astype(int)

#? Floor:

#? Exterior: binary column (exterior/interior)

#? Elevator, garage, storage_room, balcony: we will transform these columns into binary ones and fill the null values with False, 
#? as it means that the house does not have these features
for col in ['garage', 'storage_room', 'balcony', 'elevator']:
    data[col] = data[col].notnull()
    data[col] = data[col].fillna(False)

#? New: there are just two possible values: "Obra nueva" and "Segunda Mano", so we will transform this column into a binary one
# Also, the new column, as there are just 10 null values, we will check them manually and fill them with the correct value
print(data[data['new'].isnull()]["url"])
data['new'] = data['new'].map({"Obra nueva": True, "Segunda mano": False})


# After checking the urls, we see that all the houses are new, so we fill the null values with True
data['new'] = data['new'].fillna(True)


# Print the value counts of the new column to check that there are no null values
print(data['new'].value_counts())

#? Condition

#? Year: as there are many null values (45%), we will not do anything for now

#? Heating:

#? Agency: in the case of null values, it means that the house is being sold by the owner, so we will fill the null values with "owner"
data['agency'] = data['agency'].fillna("owner")
data['agency'] = data['agency'].astype('category')

#? Consumption value and label:

#? Emisions value and label:

#? Description: in the case of null values, it means that there is no description, so we will fill the null values with an empty string
data['description'] = data['description'].fillna("")


print(data.info())

# We check the percentage of null values per column
null_percentage = data.isnull().mean().sort_values(ascending=False)
print(null_percentage)

<class 'pandas.core.frame.DataFrame'>
Index: 1417 entries, 0 to 1489
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                1417 non-null   object 
 1   price              1417 non-null   object 
 2   zone               1417 non-null   object 
 3   neighborhood       1417 non-null   object 
 4   built_area         1417 non-null   float64
 5   usable_area        968 non-null    float64
 6   bedrooms           1407 non-null   float64
 7   bathrooms          1415 non-null   float64
 8   floor              1271 non-null   float64
 9   exterior           1249 non-null   object 
 10  elevator           1392 non-null   object 
 11  garage             371 non-null    object 
 12  storage_room       381 non-null    object 
 13  balcony            780 non-null    object 
 14  new                1407 non-null   object 
 15  condition          1286 non-null   object 
 16  year               759 non-nu

C:\Users\david.r\AppData\Local\Temp\ipykernel_27468\345348535.py:53: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['new'] = data['new'].fillna(True)


In [6]:
# Check the amount of houses with exterior orientation
print(data['exterior'].value_counts())

# Print the urls of the houses with null orientation
print(data[data['exterior'].isnull()]["url"])

print(data[data["new"] == True]["exterior"].isnull().sum())


exterior
exterior    1061
interior     183
Name: count, dtype: int64
10      https://www.idealista.com/inmueble/108574469/
14      https://www.idealista.com/inmueble/104105241/
15      https://www.idealista.com/inmueble/108376976/
16      https://www.idealista.com/inmueble/106596850/
24      https://www.idealista.com/inmueble/109237730/
                            ...                      
1439    https://www.idealista.com/inmueble/108638950/
1457    https://www.idealista.com/inmueble/108812275/
1462    https://www.idealista.com/inmueble/108493769/
1485    https://www.idealista.com/inmueble/107389969/
1486    https://www.idealista.com/inmueble/101540464/
Name: url, Length: 163, dtype: object
40


In [7]:
# Check the amount of houses per condition
print(data['condition'].value_counts())

condition
Buen estado    967
A reformar     309
Name: count, dtype: int64


In [8]:
# Check the amount of heating 
print(data['heating'].value_counts())

# Check the amount of houses with missing value in year, and compare each case with the value of the "new" column. We want to see how many of them have "new" as True
heating_data = data[data['heating'].notnull()][['heating', 'new']]

# We check how many of them have "new" as True
print(heating_data['new'].value_counts())

data[data["new"] == True]["heating"].value_counts()


heating
Individual    763
Central       242
Name: count, dtype: int64
new
False    968
True      37
Name: count, dtype: int64


heating
Individual    26
Central       11
Name: count, dtype: int64

In [9]:
# Check the amount of houses with missing value in year, and compare each case with the value of the "new" column. We want to see how many of them have "new" as True
year_data = data[data['year'].isnull()][['year', 'new']]

# We check how many of them have "new" as True
print(year_data['new'].value_counts())

# Print the new column
print(data['new'].value_counts())

new
False    518
True     121
Name: count, dtype: int64
new
False    1276
True      121
Name: count, dtype: int64


CONDITION CLEANING (DAVID)

In [ ]:
print(data.shape)
print("Numero Condition null: ", data['condition'].isnull().sum())


#Los 131 casos en los que "condition" es null porque la casa es NUEVA:
print("Numero Condition null y New True: ", data[(data['condition'].isnull()) & (data['new'] == True)].shape[0])

#Voy a confirmarlo a mano: 
print(data[data['condition'].isnull()]['url'])
#Efectivamente todas los valores de "condition" que son null, son casas nuevas. 

# Por lo que voy a rellenar esos 131 valores con de la columna "condition" con "Nuevo"
data.loc[data['condition'].isnull(), 'condition'] = 'Nuevo'


print(data.head())

(1407, 24)
Numero Condition null:  0
Numero Condition null y New True:  0
Series([], Name: url, dtype: object)
                                             url     price    zone  \
0  https://www.idealista.com/inmueble/109356873/  440000.0  deusto   
1  https://www.idealista.com/inmueble/106221410/  442000.0  deusto   
2  https://www.idealista.com/inmueble/107750109/  381000.0  deusto   
3  https://www.idealista.com/inmueble/106221526/  575000.0  deusto   
4  https://www.idealista.com/inmueble/108491309/  306000.0  deusto   

                      neighborhood  built_area  usable_area  bedrooms  \
0  La Ribera-Ibarrekolanda, Bilbao        76.0         70.0         2   
1  La Ribera-Ibarrekolanda, Bilbao        82.0          NaN         2   
2  La Ribera-Ibarrekolanda, Bilbao        58.0          NaN         1   
3  La Ribera-Ibarrekolanda, Bilbao       104.0          NaN         3   
4  La Ribera-Ibarrekolanda, Bilbao        62.0          NaN         1   

   bathrooms  floor  exterior

C:\Users\david.r\AppData\Local\Temp\ipykernel_27468\1137459095.py:11: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
C:\Users\david.r\AppData\Local\Temp\ipykernel_27468\1137459095.py:11: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


## STORE